## hyperband tuner

This is (hopefully) the final model tuner in this project. It should produce models with an acceptable accuracy for production.

### Please ensure that the data augmentation with suitable augmentation parameters process has been run and there are *train.tfrecord*, *validation.tfrecord* and *test.tfrecord* files in *data/processed*. 

In [1]:
from os.path import join
import tensorflow as tf

from src.image_handling import decode_record

processed = join('data', 'processed')

features = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
}
shape = (32, 32, 1)

train_dataset = decode_record(join(processed, 'train.tfrecord'), features, shape)
validation_dataset = decode_record(join(processed, 'validation.tfrecord'), features, shape)
test_dataset = decode_record(join(processed, 'test.tfrecord'), features, shape)

In [2]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adadelta, Adam, RMSprop, SGD

def create_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(2**hp.Int('2**num_filter_0', 4, 6),
        (4,4) ,activation='relu', input_shape=(32, 32, 1)))

    for i in range(hp.Int('num_cnn_layers', 0, 3)):
        filter = 2**hp.Int('2**num_filter_' + str(i), 4, 7)
        model.add(layers.Conv2D(filter, (4,4), activation='relu', padding='same'))
        if hp.Boolean('pooling_' + str(i)):
            model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Flatten())
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        nodes = 2**hp.Int('2**num_nodes_' + str(i), 4, 7)
        model.add(layers.Dense(nodes, activation='relu'))
    
    model.add(layers.Dense(3, 'softmax'))

    optimizers = {
        'adam': Adam(),
        'sgd': SGD(lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01, 0.03]),
            momentum=hp.Float('momentum', 0.6, 1, 0.1),
            nesterov=hp.Boolean('nesterov')),
        'rms': RMSprop(lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01, 0.03]))
    }

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=optimizers[hp.Choice('optimizer', list(optimizers.keys()))],
        metrics=['acc'])

    return model

In [3]:
from datetime import datetime
from tensorboard.plugins.hparams import api
from kerastuner import Hyperband
from tensorflow import summary
from tensorflow.keras.callbacks import TensorBoard

class customTuner(Hyperband):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def on_trial_end(self, trial):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the hparams where the metrics of tensorboard are
        hparam_dir = join(trial_dir, trial.trial_id)
        hparams = trial.hyperparameters.values
        with summary.create_file_writer(hparam_dir).as_default():
            api.hparams(hparams, trial_id=trial.trial_id)

        print(datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
        
        super().on_trial_end(trial)

    def on_epoch_end(self, trial, model, epoch, logs):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the data where the metrics of tensorboard are
        hist_dir = join(trial_dir, trial.trial_id)
        with summary.create_file_writer(hist_dir).as_default():
            for layer in model.weights:
                summary.histogram(layer.name, data=layer, step=epoch)
        super().on_epoch_end(trial, model, epoch, logs)

In [4]:
from kerastuner import HyperParameters

hp=HyperParameters()
log_dir = join('logs', 'srp19')
timestamp = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")

tuner = customTuner(
    create_model,
    hyperparameters=hp,
    objective='acc',
    max_epochs=50,
    factor=3,
    executions_per_trial=1,
    directory=log_dir,
    project_name=timestamp)

tuner.search_space_summary()

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [ EarlyStopping(monitor='loss', patience=3) ]

tuner.search(
    train_dataset,
    validation_data=validation_dataset,
    epochs=30,
    steps_per_epoch=100,
    validation_steps=100,
    verbose=0,
    callbacks=callbacks)

2019-11-21T03-24-04
Remaining Trials: None
2019-11-21T03-24-09
Remaining Trials: None
2019-11-21T03-24-11
Remaining Trials: None
2019-11-21T03-24-14
Remaining Trials: None
2019-11-21T03-24-19
Remaining Trials: None
2019-11-21T03-24-22
Remaining Trials: None
2019-11-21T03-24-25
Remaining Trials: None
2019-11-21T03-24-28
Remaining Trials: None
2019-11-21T03-24-32
Remaining Trials: None
2019-11-21T03-24-35
Remaining Trials: None
2019-11-21T03-24-38
Remaining Trials: None
2019-11-21T03-24-40
Remaining Trials: None
2019-11-21T03-24-44
Remaining Trials: None
2019-11-21T03-24-47
Remaining Trials: None
2019-11-21T03-24-51
Remaining Trials: None
2019-11-21T03-24-53
Remaining Trials: None
2019-11-21T03-24-57
Remaining Trials: None
2019-11-21T03-25-00
Remaining Trials: None
2019-11-21T03-25-03
Remaining Trials: None
2019-11-21T03-25-07
Remaining Trials: None
2019-11-21T03-25-12
Remaining Trials: None
2019-11-21T03-25-16
Remaining Trials: None
2019-11-21T03-25-19
Remaining Trials: None
2019-11-21T

In [6]:
tuner.results_summary()

In [14]:
test_winner = tuner.get_best_models(num_models=10)

for winner in test_winner:
    print(winner.evaluate(test_dataset, steps=100, verbose=0))

[0.6909906071305386, 0.9271875]
[0.45963708525057884, 0.9465625]
[0.3662359217298217, 0.9284375]
[0.5352356510609388, 0.884375]
[0.4679211878590286, 0.896875]
[0.3167982506752014, 0.9053125]
[0.17056896969676016, 0.946875]
[0.2715273393969983, 0.9196875]
[0.13959105822723358, 0.9634375]
[0.3617020515166223, 0.9190625]
[0.2598431269498542, 0.939375]
[0.2811727066151798, 0.926875]
[0.2390282751712948, 0.9428125]
[0.9060272057194015, 0.9053125]
[0.6114441955089569, 0.9046875]
[0.14776341523975134, 0.9509375]
[0.3063883475090552, 0.946875]
[0.2469565454078838, 0.929375]
[0.6227786477282643, 0.88]
[0.4113431348092854, 0.9109375]
